# ShohnomaLLM - Обучение модели

Fine-tuning **Qwen3-4B** для генерации таджикских стихов.

**Почему Qwen3:**
- Qwen3-4B ≈ Qwen2.5-7B по качеству
- Лучше reasoning и multilingual
- 100+ языков включая персидский

**Оптимизации для A100:**
- ✅ Packing — 2-5x ускорение
- ✅ Batch size 16
- ✅ 4 dataloader workers

**Время: ~30-60 минут**

⚠️ Runtime → Change runtime type → **A100 GPU**

In [ ]:
# Проверка GPU
!nvidia-smi

In [ ]:
# Установка зависимостей
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes

In [ ]:
# Клонируем репозиторий ShohnomaLLM
!git clone https://github.com/Kuchizu/ShohnomaLLM.git
%cd ShohnomaLLM
!pip install -r requirements.txt -q

In [ ]:
# Подключение Google Drive (для сохранения модели)
from google.colab import drive
drive.mount('/content/drive')

# Пути
REPO_DIR = "/content/ShohnomaLLM"
DATA_DIR = f"{REPO_DIR}/data"  # Данные из репозитория
MODEL_DIR = "/content/drive/MyDrive/ShohnomaLLM/models"  # Модели в Drive

# Создаём директории
!mkdir -p {DATA_DIR}/processed {DATA_DIR}/training {MODEL_DIR}

## 1. Подготовка данных

Используем модули из репозитория

In [ ]:
import sys
sys.path.insert(0, REPO_DIR)

from training.format_dataset import DatasetFormatter
from training.config import get_config

# Загружаем конфигурацию для A100
config = get_config("colab_a100")
print(f"Модель: {config.model.base_model}")
print(f"LoRA rank: {config.lora.r}")
print(f"Batch size: {config.training.per_device_train_batch_size}")
print(f"Gradient accumulation: {config.training.gradient_accumulation_steps}")

In [ ]:
# Загрузка данных с HuggingFace (если нет локальных)
import os
import json
from tqdm import tqdm

local_data = f"{DATA_DIR}/raw/ganjoor/all_classical.jsonl"
hf_data = f"{DATA_DIR}/raw/ganjoor_hf/all_poems.jsonl"

# Если HF данных нет - скачиваем
if not os.path.exists(hf_data):
    print("Загрузка датасета с HuggingFace (119K стихов)...")
    from datasets import load_dataset
    
    # Создаём директорию
    os.makedirs(f"{DATA_DIR}/raw/ganjoor_hf", exist_ok=True)
    
    # Загружаем датасет
    dataset_hf = load_dataset("mabidan/ganjoor", split="train")
    print(f"Загружено: {len(dataset_hf)} стихов")
    
    # Инициализируем транслитератор
    from scraper.utils.transliterate import PersianToTajikTransliterator
    transliterator = PersianToTajikTransliterator()
    
    # Обрабатываем и сохраняем
    print("Транслитерация в таджикскую кириллицу...")
    with open(hf_data, 'w', encoding='utf-8') as f:
        for item in tqdm(dataset_hf, desc="Обработка"):
            text = item.get("text", "")
            if not text or len(text) < 30:
                continue
            
            text_tajik = transliterator.transliterate_poem(text)
            
            poem_data = {
                "id": f"hf_{item.get('id', 0)}",
                "poet": item.get("poet", ""),
                "text_tajik": text_tajik,
                "form": "other",
                "source": "huggingface",
            }
            json.dump(poem_data, f, ensure_ascii=False)
            f.write('\n')
    
    print(f"Сохранено: {hf_data}")

# Подготовка датасета
from training.format_dataset import prepare_full_dataset

stats = prepare_full_dataset(
    raw_dir=f"{DATA_DIR}/raw",
    processed_dir=f"{DATA_DIR}/processed",
    training_dir=f"{DATA_DIR}/training",
)
print(f"\nВсего подготовлено: {stats.get('total', 0)} примеров")

## 2. Загрузка модели

In [ ]:
from unsloth import FastLanguageModel
import torch

# Загрузка модели
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=config.model.base_model,
    max_seq_length=config.model.max_seq_length,
    dtype=None,
    load_in_4bit=config.model.load_in_4bit,
)

print(f"Модель загружена: {config.model.base_model}")

In [ ]:
# Добавляем LoRA (БЕЗ gradient checkpointing для скорости на A100)
model = FastLanguageModel.get_peft_model(
    model,
    r=config.lora.r,
    target_modules=config.lora.target_modules,
    lora_alpha=config.lora.lora_alpha,
    lora_dropout=config.lora.lora_dropout,
    bias=config.lora.bias,
    use_gradient_checkpointing=False,  # Выключено для A100 - быстрее!
    random_state=42,
)

print("LoRA добавлен")
model.print_trainable_parameters()

## 3. Обучение

In [ ]:
from datasets import load_dataset

# Загрузка датасета
dataset = load_dataset(
    'json',
    data_files={
        'train': f"{DATA_DIR}/training/train.jsonl",
        'validation': f"{DATA_DIR}/training/val.jsonl",
    }
)

print(f"Train: {len(dataset['train'])}")
print(f"Val: {len(dataset['validation'])}")

In [ ]:
# Настройка параметров - МАКСИМАЛЬНАЯ СКОРОСТЬ ДЛЯ A100
from trl import SFTTrainer, SFTConfig

dataset_size = len(dataset['train'])
print(f"Размер датасета: {dataset_size}")

# Эпохи в зависимости от размера данных
if dataset_size > 30000:
    num_epochs = 2
    eval_steps = 1000
    save_steps = 1000
else:
    num_epochs = 3
    eval_steps = 500
    save_steps = 500

print(f"Epochs: {num_epochs}")
print(f"С packing ожидается ~50 минут обучения")

# Trainer с PACKING - 2-5x быстрее для коротких текстов!
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    
    args=SFTConfig(
        output_dir="./outputs",
        dataset_text_field="text",  # Поле с текстом!
        
        # === PACKING - ГЛАВНАЯ ОПТИМИЗАЦИЯ ===
        packing=True,
        max_seq_length=1024,
        dataset_num_proc=4,
        
        # === BATCH SIZE для A100 ===
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        gradient_accumulation_steps=2,
        
        # === Learning Rate ===
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        warmup_ratio=0.05,
        num_train_epochs=num_epochs,
        
        # === Precision ===
        bf16=True,
        optim="adamw_8bit",
        weight_decay=0.01,
        max_grad_norm=1.0,
        
        # === Dataloader ===
        dataloader_num_workers=4,
        dataloader_pin_memory=True,
        
        # === Logging ===
        logging_steps=50,
        eval_steps=eval_steps,
        eval_strategy="steps",
        save_steps=save_steps,
        save_strategy="steps",
        save_total_limit=2,
        load_best_model_at_end=True,
        
        seed=42,
        report_to="none",
    ),
)

In [ ]:
# Запуск обучения
print("Начало обучения...")
trainer.train()

## 4. Сохранение модели

In [ ]:
# Сохраняем LoRA
lora_path = f"{MODEL_DIR}/tajik-poetry-lora"
model.save_pretrained(lora_path)
tokenizer.save_pretrained(lora_path)
print(f"LoRA сохранён: {lora_path}")

In [ ]:
# Объединяем LoRA с базовой моделью (16-bit)
merged_path = f"{MODEL_DIR}/tajik-poetry-4b"

model.save_pretrained_merged(
    merged_path,
    tokenizer,
    save_method="merged_16bit",
)
print(f"Merged модель: {merged_path}")

## 5. Тестирование

In [ ]:
# Режим inference
FastLanguageModel.for_inference(model)

# Системный промпт
from training.format_dataset import SYSTEM_PROMPT

def generate_poem(prompt, max_tokens=256, temperature=0.8):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": prompt},
    ]
    
    text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=temperature,
        top_p=0.9,
        repetition_penalty=1.1,
        do_sample=True,
    )
    
    return tokenizer.decode(
        outputs[0][inputs.input_ids.shape[1]:],
        skip_special_tokens=True,
    )

In [ ]:
# Тестовые промпты
test_prompts = [
    "Рубоӣ бинавис",
    "Ғазали ошиқона эҷод кун",
    "Шеър дар бораи баҳор бинавис",
]

for prompt in test_prompts:
    print(f"\n{'='*50}")
    print(f"Запрос: {prompt}")
    print(f"{'='*50}")
    print(generate_poem(prompt))

## 6. Экспорт в GGUF (опционально)

Для запуска на CPU через llama.cpp

In [ ]:
# Экспорт в GGUF
gguf_path = f"{MODEL_DIR}/tajik-poetry-q4"

model.save_pretrained_gguf(
    gguf_path,
    tokenizer,
    quantization_method="q4_k_m",
)
print(f"GGUF сохранён: {gguf_path}")

In [ ]:
print("\n" + "="*50)
print("Обучение завершено!")
print("="*50)
print(f"\nМодели сохранены в Google Drive: {MODEL_DIR}")
print("\nСкачайте модель и используйте локально:")
print("  python -m cli.generate --model путь/к/модели")